# 分析 final_solutions.csv ，通过代码作图，显示三维 Pareto Front

**独立代码块**

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

data = np.loadtxt("final_solutions.csv", delimiter=",")

# data.shape = (N, 15)
# 假设前 12 列是决策变量 X，后 3 列是目标值 F。
X = data[:, :12]   # (N, 12)
F = data[:, 12:]   # (N, 3)

# 这里假设 F 里的三列已经是我们要画的数值(若需反转/还原，请在此之前做)
f0 = F[:, 0]
f1 = F[:, 1]
f2 = F[:, 2]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(f0, f1, f2, c='b', marker='o', depthshade=False)

ax.set_xlabel("Objective 1 (F0)")
ax.set_ylabel("Objective 2 (F1)")
ax.set_zlabel("Objective 3 (F2)")
plt.title("Pareto Front in 3D")

plt.show()


# 读取 res_history 文件，分析代际间适应度优化的趋势
**独立代码块**

**以单目标为例: 逐代计算最小值、平均值**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = np.load("res_history.npz", allow_pickle=True)
final_X = data["final_X"]  # (n_nd, n_var), 最终非支配解/最优解
final_F = data["final_F"]  # (n_nd, n_obj)
history_F = data["history_F"]  # (n_gen,) 个元素, 每个元素 shape=(pop_size, n_obj)

# 打印信息:
print("final_X.shape:", final_X.shape)
print("final_F.shape:", final_F.shape)
print("历代数量 = ", len(history_F))

# 以单目标为例: 逐代计算最小值、平均值
best_per_gen = []
avg_per_gen = []
for gen_idx, F_gen in enumerate(history_F):
    best_val = np.min(F_gen[:, 1])   # 第1列目标(从0开始计数), shape=(pop_size,)
    avg_val  = np.mean(F_gen[:, 1])
    best_per_gen.append(best_val)
    avg_per_gen.append(avg_val)

# 绘制
gen_axis = np.arange(1, len(history_F)+1)
plt.figure()
plt.plot(gen_axis, best_per_gen, label="Best of Gen")
plt.plot(gen_axis, avg_per_gen, label="Mean of Gen")
plt.xlabel("Generation")
plt.ylabel("Objective Value")
plt.title("Convergence Over Generations")
plt.legend()
plt.show()


In [ ]:
# 加载存储的文件
data = np.load("res_history.npz", allow_pickle=True)
print(data.files)  # ['final_X', 'final_F', 'history_F']

final_X = data["final_X"]
final_F = data["final_F"]
history_F = data["history_F"]  # 这是一个 list，每一项是每代的 F

# 可以计算并绘制最优解、平均解的变化
import matplotlib.pyplot as plt
best_per_gen = []
avg_per_gen = []

for F_gen in history_F:
    best_per_gen.append(np.min(F_gen[:, 0]))  # 单目标最小值
    avg_per_gen.append(np.mean(F_gen[:, 0]))  # 单目标平均值

# 绘制收敛图
plt.figure()
plt.plot(range(1, len(history_F)+1), best_per_gen, label="Best of Gen")
plt.plot(range(1, len(history_F)+1), avg_per_gen, label="Average of Gen")
plt.xlabel("Generation")
plt.ylabel("Objective Value")
plt.title("Convergence Over Generations")
plt.legend()
plt.show()


# 每一代（generation）和非支配解（nondominated solutions）文件，分析判断算法的优化趋势与收敛性
**独立代码块，需要自定义chkpt_dir与n_gen**

In [ ]:
%matplotlib widget
import numpy as np
import math
import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# 指定一个支持中文的字体，比如 Windows 上常见的 "SimHei" (黑体)
matplotlib.rcParams["font.sans-serif"] = ["SimHei"]  
matplotlib.rcParams["axes.unicode_minus"] = False    # 正常显示负号
from pymoo.indicators.hv import HV

chkpt_dir = r"F:\ResearchMainStream\0.ResearchBySection\C.动力学模型\参数优化\参数优化实现\ParallelSweepSimpack\结果分析组\0210早-nsga2-180群150代-收敛"  # 保存 generation_xxx.npz 的目录
n_gen = 150            # 总迭代轮次
n_obj = 3             # 你的目标函数个数

# 存储每一代的整个种群目标值（适应度函数）
all_F_history = []  # 这是一个列表，元素为 F_gen 数组 (pop_size, n_obj)

for gen in range(1, n_gen+1):
    filename = os.path.join(chkpt_dir, f"generation_{gen}.npz")
    data = np.load(filename)
    F_gen = data["F"]  # 形如 (pop_size, n_obj)
    all_F_history.append(F_gen)

# 现在 all_F_history[0] 对应第1代的所有目标值 (pop_size, n_obj),
# all_F_history[1] 对应第2代的所有目标值, ... 依次类推。

# 对于每一代，计算 F_gen 在所有个体上的最小值(逐个目标)
best_per_gen = np.zeros((n_gen, n_obj))  # shape = (88, 3)

for i in range(n_gen):
    F_gen = all_F_history[i]  # (pop_size, n_obj)
    # 计算 3 个目标各自的最小值
    best_per_gen[i, :] = F_gen.min(axis=0)

# 绘制收敛图
plt.figure(figsize=(6,4))
gen_axis = np.arange(1, n_gen+1)  # x轴为第1到第88代
for j in range(n_obj):
    plt.plot(gen_axis, best_per_gen[:, j], label=f"F{j}的最小值")
plt.xlabel("迭代代数 (generation)")
plt.ylabel("目标函数值 (Objective value)")
plt.title("每代最优目标值的变化曲线")
plt.legend()
plt.grid(True)
plt.show()

# 每一代，计算 F_gen 平均值
avg_per_gen = np.zeros((n_gen, n_obj))  # 用于存储每一代 3 个目标的平均值

for i in range(n_gen):
    F_gen = all_F_history[i]
    avg_per_gen[i, :] = F_gen.mean(axis=0)  # 计算该代目标值的均值

# 绘制平均值随迭代的变化
gen_axis = np.arange(1, n_gen + 1)  # [1, 2, ..., 88]
plt.figure(figsize=(6,4))
for j in range(n_obj):
    plt.plot(gen_axis, avg_per_gen[:, j], label=f"F{j}平均值")

plt.xlabel("迭代代数 (generation)")
plt.ylabel("目标函数平均值")
plt.title("每代目标平均值趋势")
plt.legend()
plt.grid(True)
plt.show()

fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111, projection='3d')

for gen in [1, n_gen]:  # 只示例第1代和最后一代
    filename_nd = os.path.join(chkpt_dir, f"generation_{gen}_nondom.npz")
    data_nd = np.load(filename_nd)
    F_nd = data_nd["F"]  # (n_nd, n_obj) 这里 n_obj=3
    ax.scatter(F_nd[:,0], F_nd[:,1], F_nd[:,2], depthshade=False,  label=f"Gen {gen} ND")

ax.set_xlabel("目标1 (F0)")
ax.set_ylabel("目标2 (F1)")
ax.set_zlabel("目标3 (F2)")
ax.legend()
ax.set_title("各代非支配解的 3D 散点分布")
plt.show()

# 高维指标：超体积 (Hypervolume) 或 IGD
# 在多目标优化中，衡量优化质量和收敛性的常用指标还包括 超体积（hypervolume, HV）、GD/IGD（Generational Distance / Inverted Generational Distance）等。
# 通过计算每一代的非支配解的超体积，可以看到随着迭代次数增加，超体积是否单调增加并逐渐收敛。
# 这类指标能够更全面地衡量多目标收敛和分布情况，尤其是当目标数大于 2 或 3 时更为重要。
# 如果需要计算超体积，pymoo 自带了一些指标工具，可以参考 pymoo文档 或者自行使用常见的 HV 算法包。

MAXoverUB_F1 = 0
MAXoverUB_F2 = 2000
MAXoverUB_F3 = math.sqrt( (3 * 3 + 3 * 3) / 2 ) * 100 # 由约束定义上限

# 设定一个参考点 (reference point)，大于或等于所有解的最大目标值
ref_point = np.array([MAXoverUB_F1, MAXoverUB_F2, MAXoverUB_F3])

hv_indicator = HV(ref_point=ref_point)

hv_values = []
for gen in range(1, n_gen+1):
    filename_nd = os.path.join(chkpt_dir, f"generation_{gen}_nondom.npz")
    data_nd = np.load(filename_nd)
    F_nd = data_nd["F"]  # (n_nd, 3)
    hv = hv_indicator.do(F_nd)
    hv_values.append(hv)

# hv_values[i] 就是第 i 代的超体积
fig = plt.figure()
plt.plot(range(1, n_gen+1), hv_values, marker='o')
plt.xlabel("迭代代数 (generation)")
plt.ylabel("Hypervolume")
plt.title("超体积收敛曲线")
plt.show()

# 将 NSGA-2 和 NSGA-3 的非支配解合并，重新计算它们在一起时的整体非支配前沿（ND），并用不同颜色区分它们各自的解

**独立代码块，需要自定义chkpt_dir**

文件名按照以下方式重命名：
 - generation_2_nondom.npz 存放 NSGA-2 第 2 代的非支配解
 - generation_3_nondom.npz 存放 NSGA-3 第 3 代的非支配解
 - 将一个 generation_1_nondom.npz 作为 NSGA-2 第 1 代的非支配解（示例中如果不需要，可以自行去掉此部分）

In [ ]:
%matplotlib widget

import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
# 指定一个支持中文的字体，比如 Windows 上常见的 "SimHei" (黑体)
matplotlib.rcParams["font.sans-serif"] = ["SimHei"]  
matplotlib.rcParams["axes.unicode_minus"] = False    # 正常显示负号

chkpt_dir = r"F:\ResearchMainStream\0.ResearchBySection\C.动力学模型\参数优化\参数优化实现\ParallelSweepSimpack\结果分析组\NSGA23算法效能对比"

# =========== 1. 读取 NSGA-2 / NSGA-3 各自的非支配解 ===========
# 假设:
#   - generation_1_nondom.npz 是 NSGA-2 第 1 代的非支配解
#   - generation_2_nondom.npz 是 NSGA-2 第 2 代的非支配解
#   - generation_3_nondom.npz 是 NSGA-3 第 3 代的非支配解
# 如果你只想对比某几代，也可以只读那几份文件。

# ---------- 读取 NSGA-2 的非支配解 ----------
filename_nsga2_gen1 = os.path.join(chkpt_dir, "generation_1_nondom.npz")
data_nsga2_gen1 = np.load(filename_nsga2_gen1)
F_nsga2_gen1 = data_nsga2_gen1["F"]  # (n_nd1, n_obj)
X_nsga2_gen1 = data_nsga2_gen1["X"]

filename_nsga2_gen2 = os.path.join(chkpt_dir, "generation_2_nondom.npz")
data_nsga2_gen2 = np.load(filename_nsga2_gen2)
F_nsga2_gen2 = data_nsga2_gen2["F"]  # (n_nd2, n_obj)
X_nsga2_gen2 = data_nsga2_gen2["X"]

# ---------- 读取 NSGA-3 的非支配解 ----------
filename_nsga3_gen3 = os.path.join(chkpt_dir, "generation_3_nondom.npz")
data_nsga3_gen3 = np.load(filename_nsga3_gen3)
F_nsga3_gen3 = data_nsga3_gen3["F"]  # (n_nd3, n_obj)
X_nsga3_gen3 = data_nsga3_gen3["X"]

# =========== 2. 合并解并标记其来源 (NSGA-2 or NSGA-3) ===========
# 假设 n_obj = 3
F_all = np.concatenate([F_nsga2_gen1, F_nsga2_gen2, F_nsga3_gen3], axis=0)
X_all = np.concatenate([X_nsga2_gen1, X_nsga2_gen2, X_nsga3_gen3], axis=0)

# 给每个解打一个标签，用于区分它是 NSGA-2 或 NSGA-3：
#  - 这里用数字 2 表示 NSGA-2，数字 3 表示 NSGA-3。
labels_nsga2_gen1 = np.array([2]*len(F_nsga2_gen1))
labels_nsga2_gen2 = np.array([2]*len(F_nsga2_gen2))
labels_nsga3_gen3 = np.array([3]*len(F_nsga3_gen3))

labels_all = np.concatenate([labels_nsga2_gen1, labels_nsga2_gen2, labels_nsga3_gen3], axis=0)

# =========== 3. 对合并后的所有解做一次新的非支配排序 ===========
nd_front = NonDominatedSorting().do(F_all, only_non_dominated_front=True)

# nd_front 是索引数组, 指示了合并后的 F_all 中哪些是非支配解
F_nd_all = F_all[nd_front]
X_nd_all = X_all[nd_front]
labels_nd_all = labels_all[nd_front]  # 同步提取它们的来源标签

# =========== 4. 分别取出 NSGA-2 和 NSGA-3 在此合并 ND 中的解 ===========
mask_nsga2 = (labels_nd_all == 2)
mask_nsga3 = (labels_nd_all == 3)

F_nd_nsga2 = F_nd_all[mask_nsga2]
F_nd_nsga3 = F_nd_all[mask_nsga3]

# =========== 5. 绘制散点图 ===========

fig = plt.figure(figsize=(7, 5))
ax = fig.add_subplot(111, projection='3d')

# 为了让效果更明显，这里演示一个简单的坐标变换
# 例如将 F0 用 -F0*3.6（负号是为了让“越大越好”排布更直观，或根据自己需求修正），
# F1 不变，F2 做除以200 等等
def transform(F):
    return (-F[:, 0]*3.6, F[:, 1], F[:, 2]/200)

# 绘制 NSGA-2（合并后ND）
if len(F_nd_nsga2) > 0:
    x2, y2, z2 = transform(F_nd_nsga2)
    ax.scatter(x2, y2, z2, c='red', marker='o', label='NSGA-2 ND')

# 绘制 NSGA-3（合并后ND）
if len(F_nd_nsga3) > 0:
    x3, y3, z3 = transform(F_nd_nsga3)
    ax.scatter(x3, y3, z3, c='blue', marker='^', label='NSGA-3 ND')

ax.set_xlabel("临界速度 km/h (F0)")
ax.set_ylabel("磨耗数 (F1)")
ax.set_zlabel("Sperling指标 (F2)")
ax.set_title("合并后非支配解 (ND) 的 3D 散点分布比较")
ax.legend()
plt.show()


# 可视化 Poisson 重建后的网格

**可独立运行, 需要定义非支配前沿解 .npz 文件所在的文件夹 chkpt_dir**

**将前沿解散点 F_nd 和 Poisson 网格顶点 vertices 合并后，基于它们的整体最小、最大值来设置 xlim/ylim/zlim， 保证散点与网格都在可见范围内。**

In [ ]:
%matplotlib widget

import os
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D

# ========== 读取并加载 NSGA-2 非支配解数据 ==========
chkpt_dir = r"F:\ResearchMainStream\0.ResearchBySection\C.动力学模型\参数优化\参数优化实现\ParallelSweepSimpack\结果分析组\前沿解的包覆"
gen = 81
filename_nd = os.path.join(chkpt_dir, f"generation_{gen}_nondom.npz")
data_nd = np.load(filename_nd)
F_nd = data_nd["F"]  # (n_nd, 3) 数组，每行是 (F0, F1, F2)

# ========== 1) 创建并预处理点云对象 ==========
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(F_nd)

# 基于近邻点的 PCA (Principal Component Analysis) 估计法向量 (normals)
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))

# 设置法线方向保持一致 (如一致向外或向内)
pcd.orient_normals_consistent_tangent_plane(k=30)

# ========== 2) 进行 Poisson (泊松) 曲面重建 ==========
mesh_poisson, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcd,
    depth=8,    # 八叉树深度，影响重建网格的精细程度
    width=0, 
    scale=1.1, 
    linear_fit=False
)
# (可选) 根据 densities 做过滤，去除无效面片
# ...

# ========== 3) 将 TriangleMesh 转换为 numpy 数组供 Matplotlib 显示 ==========
vertices = np.asarray(mesh_poisson.vertices)   # 形状 (N, 3)
triangles = np.asarray(mesh_poisson.triangles) # 形状 (M, 3)

# ========== 4) 使用 Matplotlib 进行可视化 ==========
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111, projection='3d')

# (a) 绘制散点
sc_points = ax.scatter(
    F_nd[:,0], F_nd[:,1], F_nd[:,2],
    depthshade=False, c='r', marker='o', s=40,
    alpha=0.8, label=f"Gen {gen} ND"
)

# (b) 绘制 Poisson 网格
mesh_surf = ax.plot_trisurf(
    vertices[:,0],
    vertices[:,1],
    vertices[:,2],
    triangles=triangles,
    cmap='viridis',       # 可选颜色映射
    linewidth=0.1,        # 线宽
    edgecolor='g',        # 线框颜色，比如黑色 'k'
    alpha=0.2             # 透明度
)

ax.set_xlabel("F0")
ax.set_ylabel("F1")
ax.set_zlabel("F2")
ax.set_title("ND Points + Poisson Surface")

# ========== 5) 根据「前沿解 + 网格顶点」的整体坐标范围，设定轴范围 ==========
#  将两个数组拼接起来，一次性获取 x/y/z 的最小/最大值
all_coords = np.vstack((F_nd, vertices))
# 如果 all_coords = np.vstack((F_nd,)), 则最终显示视角反而较为局限

x_min, x_max = all_coords[:,0].min(), all_coords[:,0].max()
y_min, y_max = all_coords[:,1].min(), all_coords[:,1].max()
z_min, z_max = all_coords[:,2].min(), all_coords[:,2].max()

# (可选) 加一点边界空隙
margin_ratio = 0.05
x_margin = (x_max - x_min) * margin_ratio
y_margin = (y_max - y_min) * margin_ratio
z_margin = (z_max - z_min) * margin_ratio

ax.set_xlim(x_min - x_margin, x_max + x_margin)
ax.set_ylim(y_min - y_margin, y_max + y_margin)
ax.set_zlim(z_min - z_margin, z_max + z_margin)

# (可选) 手动创建图例句柄
mesh_proxy = Line2D([0],[0], marker='s', color='w', markerfacecolor='cyan',
                    label='Poisson Surface', markersize=10)
ax.legend(handles=[sc_points, mesh_proxy], loc='best')

# ========== 6) 设置初始观察角度 ==========
ax.view_init(elev=29, azim=53, roll=6)

plt.show()

## 辅助函数: 手写dominance check（原理性验证）

In [ ]:
def check_non_dominated_solutions(F):
    """
    F: shape (N, M), N是解的数量, M是目标维度
    return: indices_of_dominated, indices_of_non_dominated
    """
    N = F.shape[0]
    dominated = np.zeros(N, dtype=bool)

    for i in range(N):
        for j in range(N):
            if j == i:
                continue
            # 如果 F[j] 在所有目标上 <= F[i], 并且至少一个目标上 < F[i],
            # 那说明 F[j] 支配 F[i].
            if np.all(F[j] <= F[i]) and np.any(F[j] < F[i]):
                dominated[i] = True
                break

    indices_dominated = np.where(dominated)[0]
    indices_nondominated = np.where(~dominated)[0]
    return indices_dominated, indices_nondominated


# ========== 读取并检查例如 NSGA-2 的非支配解数据 ==========
import os
import numpy as np

chkpt_dir = r"F:\ResearchMainStream\0.ResearchBySection\C.动力学模型\参数优化\参数优化实现\ParallelSweepSimpack\结果分析组\前沿解的包覆"
gen = 81
filename_nd = os.path.join(chkpt_dir, f"generation_{gen}_nondom.npz")
F_test = np.load(filename_nd)["F"]
dom_idx, nd_idx = check_non_dominated_solutions(F_test)
print("被支配的解数量: ", len(dom_idx))
print("非支配的解数量: ", len(nd_idx))

if len(dom_idx) == 0:
    print("所有解都是非支配解！")
else:
    print("注意：有些解不应当出现在前沿解集合中！")

# Markdown 占位

In [ ]:
# 代码占位